In [1]:
import requests
from bs4 import BeautifulSoup
# Commented out from Win environment
# from scrapy.selector import Selector
import lxml
import pandas as pd
import re
import time
import random

%matplotlib inline

In [ ]:
import pandas as pd
df2 = pd.read_csv()

In [2]:
job_site = 'https://au.indeed.com'
start_page = '&start='
page_count = ''
counter = 50
store_job = []
next_button_len = 2

In [3]:
url_indeed = 'https://au.indeed.com/jobs?q=%28python+or+sql%29++%28data+or+scientist+or+science+or+analyst+or+research+or+business+or+intelligence+or+analytics+or+statistics+or+Actuarial+or+actuary+or+%22data+engineer%22%29+-developer%2C+-clerk%2C+-software%2C+-network%2C+-security%2C+-programmer&sort=date&radius=50&limit=50'
# url_indeed = 'https://au.indeed.com/jobs?as_and=%27data+scientist%27&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&as_src=&salary=&radius=100&l=Sydney&fromage=any&limit=50&sort=&psf=advsrch&from=advancedsearch'

In [4]:
ua_source_url='https://deviceatlas.com/blog/list-of-user-agent-strings#desktop'

In [5]:
def get_ua_list(source=ua_source_url):
    r = requests.get(source)
    soup = BeautifulSoup(r.content, "html.parser")
    tables = soup.find_all('table')
    return [table.find('td').text for table in tables]

In [4]:
def delay() -> None:
    time.sleep(random.uniform(15, 30))
    return None

In [40]:
random.choice(get_ua_list())

'Mozilla/5.0 (Linux; Android 7.0; SM-G930VC Build/NRD90M; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/58.0.3029.83 Mobile Safari/537.36'

In [5]:
def soup_req(url):
#     user_agent = random.choice(get_ua_list())
    req = requests.get(url)
    soup = BeautifulSoup(req.content,features='lxml')
    
    return soup

In [6]:
soup_indeed = soup_req(url_indeed)
# link_indeed = soup_indeed.find_all('div', {'class': 'title'})
link_indeed = soup_indeed.find_all('div', class_='result')

In [12]:
while next_button_len == 2:
    
    for p_job in link_indeed:
        row = {}
        join_site = [job_site, p_job.find('a')['href']]
        new_url = ''.join(join_site)
        new_soup = soup_req(new_url)

        row['job_title'] = new_soup.find('h3').text if new_soup.find('h3') else 'none'
        row['company_name'] = new_soup.find('div', attrs={'class': 'icl-u-lg-mr--sm icl-u-xs-mr--xs'}).text if new_soup.find('div', attrs={'class': 'icl-u-lg-mr--sm icl-u-xs-mr--xs'}) else 'none'
        row['location'] = new_soup.find_all(attrs={'class': 'jobsearch-JobMetadataHeader-iconLabel'})[0].text if new_soup.find_all(attrs={'class': 'jobsearch-JobMetadataHeader-iconLabel'}) else 'none'

        class_len = len(new_soup.find_all(attrs={'class': 'jobsearch-JobMetadataHeader-iconLabel'}))
        salary = 'none'
        position = 'none'

        for i in range(1,class_len):

            if i == 1:
                row['salary'] = new_soup.find_all(attrs={'class': 'jobsearch-JobMetadataHeader-iconLabel'})[1].text
                if row['salary'][0] != '$':
                    row['position'] = new_soup.find_all(attrs={'class': 'jobsearch-JobMetadataHeader-iconLabel'})[1].text
                    row['salary'] = 'none'
            elif i == 2: 
                row['salary'] = new_soup.find_all(attrs={'class': 'jobsearch-JobMetadataHeader-iconLabel'})[2].text

        row['job_desc'] = new_soup.find('div', attrs={'class': 'jobsearch-JobComponent-description'}).text if new_soup.find('div', attrs={'class': 'jobsearch-JobComponent-description'}) else 'none'

        store_job.append(row)
    
    counter += 50
    page_count = counter 
    next_page = ''.join([url_indeed, start_page, str(page_count)])
    delay()
    soup_indeed = soup_req(next_page)
    
    link_indeed = soup_indeed.find_all('div', {'class': 'title'})
    next_button_len = len(soup_indeed.find_all('span', attrs={'class': 'np'}))

In [14]:
df_job = pd.DataFrame(store_job)
df_job.to_csv('Job_final_output_v5.csv')

In [13]:
counter

950

In [16]:
next_page

'https://au.indeed.com/jobs?as_and=data+engineer&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&as_src=&salary=&radius=50&l=&fromage=any&limit=50&sort=&psf=advsrch&from=advancedsearch&start=950'

In [15]:
df_job

,job_title,company_name,location,salary,position,job_desc
0,ICT Support and Test Engineer,EHW Technology Pty Ltd,Mackay QLD,"$60,000 a year","Full-time, Permanent","Mackay QLDFull-time, Permanent$60,000 a year\n..."
1,Desktop Deployment Engineer,Muscatech,Canberra ACT,$30 - $35 an hour,"Casual, Subcontract","Canberra ACTCasual, Subcontract$30 - $35 an ho..."
2,Windows Patch Management Engineer,Chandler Macleod,Melbourne VIC,none,Contract,Melbourne VICContract\n try {\n...
3,Production Engineer,Jindex Pty Ltd,Frenchs Forest NSW 2086,"$60,000 - $69,999 a year",NaN,"Frenchs Forest NSW 2086$60,000 - $69,999 a yea..."
4,Electrical Engineering Draftsperson,UtilityVision Pty Ltd,Craigieburn VIC,$35.00 - $39.99 an hour,Permanent,Craigieburn VICPermanent$35.00 - $39.99 an hou...
...,...,...,...,...,...,...
1015,Scrum Master,PCCW Global,Brisbane QLD 4000,none,"Full-time, Permanent","Brisbane QLD 4000Full-time, Permanent\n ..."
1016,Software Process Engineer,Maptek,Adelaide SA,NaN,NaN,Adelaide SA\n try {\n ...
1017,Enterprise Sales Engineer - Singapore,Datadog,Sydney NSW,NaN,NaN,Sydney NSW\n try {\n ...
1018,C4I Communications Specialist,Nova Systems ANZ,Canberra ACT,NaN,NaN,Canberra ACT\n try {\n ...


In [6]:
job_site = 'https://www.seek.com.au/'
start_page = '&start='
page_count = ''
counter = 50
store_job = []
next_button_len = 2

In [8]:
url_seek = 'https://www.seek.com.au/data-scientist-jobs?sortmode=KeywordRelevance'

In [9]:
soup_indeed = soup_req(url_seek)

In [19]:
soup_indeed.find_all('h1').find_all('a')

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [ ]:
link_indeed = soup_indeed.find_all('div', {'class': 'title'})

In [ ]:
while next_button_len == 2:
    
    for p_job in link_indeed:
        row = {}
        join_site = [job_site, p_job.find('a')['href']]
        new_url = ''.join(join_site)
        new_soup = soup_req(new_url)

        row['job_title'] = new_soup.find('h3').text if new_soup.find('h3') else 'none'
        row['company_name'] = new_soup.find('div', attrs={'class': 'icl-u-lg-mr--sm icl-u-xs-mr--xs'}).text if new_soup.find('div', attrs={'class': 'icl-u-lg-mr--sm icl-u-xs-mr--xs'}) else 'none'
        row['location'] = new_soup.find_all(attrs={'class': 'jobsearch-JobMetadataHeader-iconLabel'})[0].text if new_soup.find_all(attrs={'class': 'jobsearch-JobMetadataHeader-iconLabel'}) else 'none'

        class_len = len(new_soup.find_all(attrs={'class': 'jobsearch-JobMetadataHeader-iconLabel'}))
        salary = 'none'
        position = 'none'

        for i in range(1,class_len):

            if i == 1:
                row['salary'] = new_soup.find_all(attrs={'class': 'jobsearch-JobMetadataHeader-iconLabel'})[1].text
                if row['salary'][0] != '$':
                    row['position'] = new_soup.find_all(attrs={'class': 'jobsearch-JobMetadataHeader-iconLabel'})[1].text
                    row['salary'] = 'none'
            elif i == 2: 
                row['salary'] = new_soup.find_all(attrs={'class': 'jobsearch-JobMetadataHeader-iconLabel'})[2].text

        row['job_desc'] = new_soup.find('div', attrs={'class': 'jobsearch-JobComponent-description'}).text if new_soup.find('div', attrs={'class': 'jobsearch-JobComponent-description'}) else 'none'

        store_job.append(row)
    
    counter += 50
    page_count = counter 
    next_page = ''.join([url_indeed, start_page, str(page_count)])
    delay()
    soup_indeed = soup_req(next_page)
    
    link_indeed = soup_indeed.find_all('div', {'class': 'title'})
    next_button_len = len(soup_indeed.find_all('span', attrs={'class': 'np'}))